## **Распределительная задача**

Завод может производить Y = 25 видов продукции. На продукцию есть покупатель, которому требуется определенное количество каждого вида:

Таблица M (кг): M = [52 37 65 64 66 66 62 49 33 54 49 67 60 68 69 35 45 53 62 40 32 54 64 60 47]

Для каждого вида продукции он либо покупает требуемое количество, либо не покупает совсем, если мы произвели недостаточно. При этом известна цена контракта на каждый вид продукции:

Таблица C (рубли): C = [656 672 675 665 512 620 451 538 523 670 591 421 574 531 608 454 471 414 605 543 612 514 638 444 437]

Завод одновременно может выпускать только один вид продукции со скоростью:

Таблица P (кг в сутки): P = [19 15 27 19 29 16 22 18 16 27 21 21 24 25 28 21 24 21 15 15 17 29 27 24 21]

Требуется написать математическую модель и найти оптимальный план производства на S = 20 суток с помощью Gurobi, если известно, что переход от одной продукции к другой происходит мгновенно, а его стоимость R = 49 рублей.


#### Запишем математическую модель:

Введём переменные:

$$x_{i} = \begin{cases}
   1,&\text{если заключаем контракт с } i &\text{заводом,}\\
   0,&\text{в противном случае.}
\end{cases}$$

Целевая функция:

$$\sum\limits_{i = 0}^{Y} (c_i - R)x_i + R\to \max_{x}$$

Ограничения:

$$x_i \in \mathbb B, \forall i \in Y $$

$$\sum\limits_{i = 0}^{Y} x_i \lceil m_i/p_i \rceil \leqslant S $$

In [108]:
import gurobipy as gb
from gurobipy import GRB
import numpy as np

In [109]:
Y = 25
S = 20
R = 49

M = np.array([52, 37, 65, 64, 66, 66, 62, 49, 33, 54, 49, 67, 60, 68, 69, 35, 45, 53, 62, 40, 32, 54, 64, 60, 47])
C = np.array([656, 672, 675, 665, 512, 620, 451, 538, 523, 670, 591, 421, 574, 531, 608, 454, 471, 414, 605, 543, 612, 514, 638, 444, 437])
P = np.array([19, 15, 27, 19, 29, 16, 22, 18, 16, 27, 21, 21, 24, 25, 28, 21, 24, 21, 15, 15, 17, 29, 27, 24, 21])

In [110]:
problem = gb.Model('Distribution_problem')

In [111]:
factory = problem.addVars(range(0, Y), vtype=gb.GRB.BINARY, name='factory')

obj = gb.LinExpr(C - R, factory.values()) + R
problem.setObjective(obj, gb.GRB.MAXIMIZE)
problem.addConstr(gb.LinExpr(np.ceil(M / P), factory.values()) <= S)

problem.update()


In [112]:
problem.optimize()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[x86])

CPU model: Intel(R) Core(TM) i5-4278U CPU @ 2.60GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 1 rows, 25 columns and 25 nonzeros
Model fingerprint: 0x32e5df54
Variable types: 0 continuous, 25 integer (25 binary)
Coefficient statistics:
  Matrix range     [2e+00, 5e+00]
  Objective range  [4e+02, 6e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+01, 2e+01]
Found heuristic solution: objective 3386.0000000
Presolve removed 1 rows and 25 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.03 seconds (0.00 work units)
Thread count was 1 (of 4 available processors)

Solution count 2: 4565 3386 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.565000000000e+03, best bound 4.565000000000e+03, gap 0.0000%


In [113]:
print(f'Заключили контракт c заводами: {[var.index + 1 for var in problem.getVars() if var.X]}')
print(f'Максимальная прибыль: {problem.objVal}')

Заключили контракт c заводами: [1, 2, 3, 10, 17, 21, 22, 23]
Максимальная прибыль: 4565.0


#### **Вопрос 1**

Ваш завод не получал новое оборудование с 80-х годов, поэтому оборудование становится непригодным каждый третий сделанный товар. На починку уходит 150. Найти оптимальный план на S суток.

In [114]:
W = 150

In [115]:
problem = gb.Model('Broken_distribution_problem')

In [116]:
factory = problem.addVars(range(0, Y), vtype=gb.GRB.BINARY, name='factory')

obj = gb.LinExpr(C - R - W * (np.ceil(M / P) // 3), factory.values()) + R
problem.setObjective(obj, gb.GRB.MAXIMIZE)
problem.addConstr(gb.LinExpr(np.ceil(M / P), factory.values()) <= S)

problem.update()


In [117]:
problem.optimize()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[x86])

CPU model: Intel(R) Core(TM) i5-4278U CPU @ 2.60GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 1 rows, 25 columns and 25 nonzeros
Model fingerprint: 0x143b98c4
Variable types: 0 continuous, 25 integer (25 binary)
Coefficient statistics:
  Matrix range     [2e+00, 5e+00]
  Objective range  [2e+02, 6e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+01, 2e+01]
Found heuristic solution: objective 2486.0000000
Presolve removed 1 rows and 25 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.04 seconds (0.00 work units)
Thread count was 1 (of 4 available processors)

Solution count 2: 3965 2486 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.965000000000e+03, best bound 3.965000000000e+03, gap 0.0000%


In [118]:
print(f'Заключили контракт c заводами: {[var.index + 1 for var in problem.getVars() if var.X]}')
print(f'Максимальная прибыль: {problem.objVal}')

Заключили контракт c заводами: [1, 2, 3, 10, 17, 21, 22, 23]
Максимальная прибыль: 3965.0


#### **Вопрос 2**

Сгенерировать 6 пар связанных контрактов, без выполнения первого в которых нельзя выполнить второй. Причём, второй элемент в паре должен быть из рекомендованных, а первый - наоборот.

In [119]:
connections = np.array([(1, 4), (2, 5), (3, 6), (10, 7), (17, 8), (21, 9)])


In [120]:
problem = gb.Model('Pair_distribution_problem')

In [121]:
factory = problem.addVars(range(0, Y), vtype=gb.GRB.BINARY, name='factory')

obj = gb.LinExpr(C - R, factory.values()) + R
problem.setObjective(obj, gb.GRB.MAXIMIZE)
problem.addConstr(gb.LinExpr(np.ceil(M / P), factory.values()) <= S)

for i, j in connections:
    problem.addConstr((factory[i] == 1) >> (factory[j] == 1))

problem.update()


In [122]:
problem.optimize()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[x86])

CPU model: Intel(R) Core(TM) i5-4278U CPU @ 2.60GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 1 rows, 25 columns and 25 nonzeros
Model fingerprint: 0x981d092a
Model has 6 general constraints
Variable types: 0 continuous, 25 integer (25 binary)
Coefficient statistics:
  Matrix range     [2e+00, 5e+00]
  Objective range  [4e+02, 6e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+01, 2e+01]
  GenCon rhs range [1e+00, 1e+00]
  GenCon coe range [1e+00, 1e+00]
Found heuristic solution: objective 49.0000000
Presolve added 6 rows and 0 columns
Presolve time: 0.00s
Presolved: 7 rows, 25 columns, 37 nonzeros
Found heuristic solution: objective 4074.0000000
Variable types: 0 continuous, 25 integer (25 binary)

Root relaxation: objective 4.461000e+03, 4 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Wor

In [123]:
print(f'Заключили контракт c заводами: {[var.index + 1 for var in problem.getVars() if var.X]}')
print(f'Максимальная прибыль: {problem.objVal}')

Заключили контракт c заводами: [1, 2, 5, 10, 17, 21, 22, 23]
Максимальная прибыль: 4402.0
